## Loend nimeüksustest

Eelnevalt eraldati lausetest NerTaggeri poolt tuvastatud nimeüksused, teostati neil esmased korrektuurid ning salvestati tulemused. Siin on eesmärgiks teostada nimeüksustele sagedusanalüüs, et tekitada loend sagedamini esinevatest isiku- ja organisatsiooninimedest. Seda loendit kasutatakse hiljem huvipakkuvate nimeüksuste uuesti lausetele peale märgendamiseks.

In [26]:
# Vajalikud impordid
from estnltk import Text
import sqlite3
import csv
from collections import Counter
import itertools

### I Andmete sisselugemine

Alustuseks loetakse CSV-failist vastavatesse järjenditesse kõik sinna eelnevalt salvestatud isiku- ja organisatsiooninimed ja nende sagedused:

In [51]:
entities_PER = []
entities_ORG = []

In [53]:
with open('ner_frequencies.csv', encoding='UTF-8') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        if row['value'] == 'PER':
            entities_PER.append(tuple([row['name'], row['freq']]))
        elif row['value'] == 'ORG':
            entities_ORG.append(tuple([row['name'], row['freq']]))

In [28]:
# Alternatiivselt saab lugeda ka andmebaasist, kuid seal puuduvad esinemissagedused
#con = sqlite3.connect("media_data_complete.db")
#cur = con.cursor()

In [22]:
#for row in cur.execute("SELECT PER FROM named_entities where PER IS NOT NULL"):
#    entities_PER.append(row[0])

In [23]:
#for row in cur.execute("SELECT ORG FROM named_entities where ORG IS NOT NULL"):
#    entities_ORG.append(row[0])

In [29]:
#con.close()

Vaatame saadud järjendeid:

In [54]:
print(entities_PER[:20])

[('hans', '15'), ('eerik', '9'), ('andrus ansip', '242'), ('meelis', '35'), ('olev esula', '1'), ('vadim', '2'), ('juskin', '1'), ('margus', '11'), ('sven', '31'), ('siim kallas', '439'), ('deniss borodits', '1'), ('jukuminek', '1'), ('borodits', '3'), ('jüri', '90'), ('eiki', '29'), ('kerviel', '2'), ('andres', '54'), ('toomas hendrik', '47'), ('meri', '36'), ('lasna', '1')]


In [55]:
print(entities_ORG[:20])

[('h', '3'), ('taska', '1'), ('vadim belov', '1'), ('pbk', '40'), ('delfi', '114'), ('tallinn ülikool', '109'), ('el', '48'), ('ccbe', '1'), ('kapo', '75'), ('irl roheline', '1'), ('sots', '60'), ('keskerakond', '2286'), ('etv', '56'), ('demokraat obama', '1'), ('err', '53'), ('irl', '1042'), ('forever', '1'), ('annepii', '1'), ('värimäe', '1'), ('res publica', '135')]


### II Saates kõnelejate nimede välja filtreerimine

Esiteks eemaldatakse nimeüksuste hulgast need nimed, mis viitavad saatejuhtidele (sageli mainitakse saate sissejuhatuses ja lõpus).

Repliikide tabelist loetakse sisse (unikaalsed) kõnelejanimed:

In [56]:
speakers = []

In [57]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [58]:
for row in cur.execute("SELECT DISTINCT person FROM lines"):
    speakers.append(row[0])

In [59]:
con.close()

In [60]:
# millised on
print(speakers[:10])

['Tundmatu3', 'Tundmatu1', 'Tundmatu2', 'Tundmatu4', 'Tundmatu5', 'Tundmatu6', 'Tundmatu8', 'Tundmatu7', 'K1', 'K2']


Nimed väiketähestatakse, sest sagedusloendis on juba väiketähestatud nimed:

In [61]:
speakers_lower = [speaker.lower() for speaker in speakers]

Seejärel käiakse läbi isikunimede sagedusloend ja filtreeritakse välja nimed, mis esinevad kõnelejanimede seas:

In [62]:
entities_PER_cleaned = [name for name in entities_PER if name[0] not in speakers_lower]

Esialgsed isiku- ja organisatsiooninimede sagedusloendid sorteeritakse kahanevas järjestuses:

In [74]:
entities_PER_sorted = sorted(entities_PER_cleaned, reverse=True, key=lambda x: int(x[1]))
entities_ORG_sorted = sorted(entities_ORG, reverse=True, key=lambda x: int(x[1]))

In [76]:
print(entities_PER_sorted[:10])
print(entities_ORG_sorted[:10])

[('edgar savisaar', '717'), ('savisaar', '648'), ('siim kallas', '439'), ('jüri ratas', '416'), ('mart luik', '334'), ('ansip', '303'), ('jürgen ligi', '253'), ('andrus ansip', '242'), ('priit', '224'), ('indrek', '214')]
[('keskerakond', '2286'), ('reformierakond', '2075'), ('riigikogu', '1540'), ('irl', '1042'), ('postimees', '594'), ('nato', '438'), ('keskpäevatund', '266'), ('eesti energia', '244'), ('tallinn sadam', '205'), ('euroopa komisjon', '195')]


### III Isikunimede tükeldamine

Esialgsest isikunimede sagedusloendist on näha, et paljudele täispikkadele isikunimedele esineb ka vasteid kas ainult eesnime või ainult perekonnanime näol (nt Edgar Savisaar ja Savisaar). Samas ei pruugi selliseid vasteid esineda kõigile täispikkadele isikunimedele käesolevas loendis. Et hilisemat nimeolemite otsingut parandada (saada rohkem tulemusi), tükeldatakse kõik täispikad nimed ees- ja perekonnanimedeks. Järgneva lähenemise puuduseks on see, et siin ei püüta eristada ees- ja perenimest koosnevaid üksuseid kahest (või enamast) tühikuga eraldatud eesnimest koosnevatest üksustest (nt Toomas Hendrik). Mitmest eesnimest koosnevaid üksuseid oleks teoorias võimalik eristada osalise edukusega, kui tekstis leidub ka koos perekonnanimega täispikk variant, mille vastu eesnimesid võrrelda.

Organisatsiooninimede peal seda lähenemist ei kasutata, sest pole alust arvata, et isikunimede kasutustavad organisatsiooninimedele laieneks. Pigem on sagedasem organisatsiooninimede puhul täispika nime asendamine lühendi/akronüümiga (nt Kaitsepolitseiamet -> KAPO). Siin praegu ei püüta lühendite/akronüümide võrdsustamist täispikkade nimedega lahendada.

Nimeüksused tükeldatakse:

In [77]:
entities_splitted = [name[0].rsplit(' ', 1) for name in entities_PER_cleaned if len(name[0].split(' ')) > 1]

Vähendatakse järjendi dimensioone:

In [79]:
entities_unnested = [name for entity in entities_splitted for name in entity]

In [80]:
print(entities_unnested[:10])

['priit', 'hõbemäg', 'rein', 'lang', 'kersti', 'kaljulaid', 'hannes', 'hanso', 'indrek', 'teder']


Saadud tükeldatud nimedest tehakse uus sagedusloend:

In [81]:
frequencies_entities_unnested = Counter(entities_unnested)

Sagedusloend viiakse samasse formaati esialgse isikunimede loendiga:

In [88]:
list_to_append = list(frequencies_entities_unnested.items())
print(list_to_append[:10])

[('priit', 46), ('hõbemäg', 3), ('rein', 49), ('lang', 3), ('kersti', 13), ('kaljulaid', 5), ('hannes', 12), ('hanso', 3), ('indrek', 28), ('teder', 4)]


Saadud järjend kleebitakse varasema isikunimede järjendi lõppu:

In [89]:
entities_PER_sorted = entities_PER_sorted + list_to_append

In [97]:
len(entities_PER_sorted)

9586

In [98]:
len(entities_ORG_sorted)

2904

In [99]:
print(len(entities_PER_sorted)+len(entities_ORG_sorted))

12490


### IV Vigade filtreerimine

Praegusel kujul võib olla jäänud alles ka siiski vigaseid nimesid, mis võivad hiljem probleeme tekitada (nt "ja"). Selle vastu võib aidata haruldaste juhtude eemaldamine enne uut nimeüksuste märgendamist, sest eeldatavsti ei ole kõige suuremad vead eriti sagedased. Alampiiriks valitakse hetkel üsna suvaliselt esinemissagedus 10. Alampiiri võib soovi korral aga määrata ka statistika abil.

In [108]:
names_to_remove = []

for name in entities_PER_sorted:
    if int(name[1]) < 10:
        names_to_remove.append(name)
for name in entities_ORG_sorted:
    if int(name[1]) < 10:
        names_to_remove.append(name)

In [109]:
print(names_to_remove[:10])

[('eerik', '9'), ('evelin', '9'), ('hannes hanso', '9'), ('ivan', '9'), ('indrek teder', '9'), ('kelly', '9'), ('tamur tsäkko', '9'), ('merko', '9'), ('maie', '9'), ('argo ideon', '9')]


In [110]:
len(names_to_remove)

11856

In [111]:
entities_PER_cleaned = [name for name in entities_PER_sorted if name not in names_to_remove]
entities_ORG_cleaned = [name for name in entities_ORG_sorted if name not in names_to_remove]

In [112]:
print(len(entities_PER_cleaned)+len(entities_ORG_cleaned))

634


### V Loendi salvestamine

Saadud loendid isiku- ja organisatsiooninimedest talletatakse csv-failis kujul, mis sobib hiljem nimeüksusi uuesti peale märgendavale taggerile ette andmiseks:

In [113]:
with open('entity_phrases.csv', 'w', newline='', encoding='UTF-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['phrase', 'value'])
    writer.writeheader()
    header2 = {'phrase': 'callable', 'value': 'string'}
    writer.writerow(header2)
    split_names_PER = [name[0].split() for name in entities_PER_cleaned]
    split_names_ORG = [name[0].split() for name in entities_ORG_cleaned]
    for name in split_names_PER:
        if len(name) > 1:
            writer.writerow({'phrase': ','.join(["'" + part + "'" for part in name]), 'value': 'PER'})
        else:
            writer.writerow({'phrase': "'" + name[0] + "'" + ",", 'value': 'PER'})
    for name in split_names_ORG:
        if len(name) > 1:
            writer.writerow({'phrase': ','.join(["'" + part + "'" for part in name]), 'value': 'ORG'})
        else:
            writer.writerow({'phrase': "'" + name[0] + "'" + ",", 'value': 'ORG'})

In [114]:
with open('entity_phrases.csv', encoding='UTF-8') as csv_file:
    rows = []
    reader = csv.DictReader(csv_file)
    for row in reader:
        rows.append(row)

In [115]:
print(rows[:10])

[{'phrase': 'callable', 'value': 'string'}, {'phrase': "'edgar','savisaar'", 'value': 'PER'}, {'phrase': "'savisaar',", 'value': 'PER'}, {'phrase': "'siim','kallas'", 'value': 'PER'}, {'phrase': "'jüri','ratas'", 'value': 'PER'}, {'phrase': "'mart','luik'", 'value': 'PER'}, {'phrase': "'ansip',", 'value': 'PER'}, {'phrase': "'jürgen','ligi'", 'value': 'PER'}, {'phrase': "'andrus','ansip'", 'value': 'PER'}, {'phrase': "'priit',", 'value': 'PER'}]


Hetkel ei ole isikunimeüksuste juurde märgitud, kas tegemist on poliitikuga või mitte. Kuna KUKU Keskpäevatund on poliitikateemaline raadiosaade võib eeldada, et suur osa (iseäranis sagedasematest) isikunimedest kuulub poliitikutele, kuid kindlasti mitte kõik. Kui soovida eristada kindlalt poliitikute nimesid mittepoliitikute omadest, tuleb teha seda käsitsi.